In [3]:
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
# Get API key from environment variable
FOURSQUARE_API_KEY = os.environ.get('FOURSQUARE_API_KEY')

# Ensure API key is available
if FOURSQUARE_API_KEY is None:
    raise ValueError("Foursquare API key not found in environment variables.")
station_df = pd.read_csv('station_details.csv')
# Define URL and parameters
all_pubs_info = []
for index, station in station_df.iterrows():
    url = "https://api.foursquare.com/v3/places/search"
    params = {
    "ll": f"{station['Latitude']},{station['Longitude']}", 
    "radius": 1000,
    "query": "pub",
    "sort": "DISTANCE"
}

# Set up headers for Authorization
headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Make request
response = requests.get(url, params=params, headers=headers)

# Check request success
if response.status_code == 200:
    data = response.json()
    pubs = data.get("results",[])
    print(data)
    
else:
    print(f"Error: {response.status_code}")
    print(response.text)


{'results': [{'fsq_id': '53d6c37d498e1ed5df675e97', 'categories': [{'id': 13018, 'name': 'Pub', 'short_name': 'Pub', 'plural_name': 'Pubs', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'short_name': 'Restaurant', 'plural_name': 'Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 71, 'geocodes': {'drop_off': {'latitude': 53.341416, 'longitude': -6.274872}, 'main': {'latitude': 53.341519, 'longitude': -6.27471}, 'roof': {'latitude': 53.341519, 'longitude': -6.27471}}, 'link': '/v3/places/53d6c37d498e1ed5df675e97', 'location': {'address': '33 Francis St', 'admin_region': 'Leinster', 'country': 'IE', 'cross_street': '', 'formatted_address': '33 Francis St, Dublin', 'locality': 'Dublin', 'postcode': 'Dublin 8'}, 'name': 'Liberty Belle, The', 'related_places': {}, 'timezone': 'Europe/Dubl

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [6]:
all_pubs_info = []
unique_pubs=set()
# Iterate through stations in DataFrame
for index, station in station_df.iterrows():
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "ll": f"{station['Latitude']},{station['Longitude']}",
        "radius": 1000,
        "query": "pubs",
        "sort": "DISTANCE",
    }

    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3yYcVFcGvMfSimEyezzk936+mryUcYCWMOxJDuvFEtOs="
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        pubs = data.get("results", [])

        for pub in pubs:
            open_status = pub.get("closed_bucket", "N/A")
            if pub.get("name") not in unique_pubs and open_status in ["VeryLikelyOpen" or "LikelyOpen"]:
                pub_info = {
                    "Station": station['Station'],
                    "Name": pub.get("name", "N/A"),
                    "Location": pub.get("location", {}).get("address", "N/A"),
                    "Open Status": pub.get("closed_bucket", "N/A"),
                }
                all_pubs_info.append(pub_info)
                unique_pubs.add(pub.get("name"))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

# Print all pub information
for pub_info in all_pubs_info:
    print(f"Pubs near {pub_info['Station']} station:")
    print(f"Name: {pub_info['Name']}")
    print(f"Location: {pub_info['Location']}")
    print(f"Open Status: {pub_info['Open Status']}\n")

Pubs near GEORGES LANE station:
Name: Delaney's
Location: 83 Nth King st
Open Status: VeryLikelyOpen

Pubs near GEORGES LANE station:
Name: The Cobblestone
Location: 77 North King St
Open Status: VeryLikelyOpen

Pubs near GEORGES LANE station:
Name: Walsh's Pub
Location: 6 Stoneybatter
Open Status: VeryLikelyOpen

Pubs near GEORGES LANE station:
Name: Oscars Café Bar
Location: 6 Smithfield Square
Open Status: VeryLikelyOpen

Pubs near GEORGES LANE station:
Name: L. Mulligan Grocer
Location: 18 Stoneybatter
Open Status: VeryLikelyOpen

Pubs near GEORGES LANE station:
Name: Tommy O'Gara's Pub
Location: 19 Stoneybatter, Dublin, Dublin 7
Open Status: VeryLikelyOpen

Pubs near GEORGES LANE station:
Name: Old Jameson Distillery
Location: Bow St, Smithfield Village
Open Status: VeryLikelyOpen

Pubs near NORTH CIRCULAR ROAD (O'CONNELL'S) station:
Name: Kavanagh Court
Location: N/A
Open Status: VeryLikelyOpen

Pubs near NORTH CIRCULAR ROAD (O'CONNELL'S) station:
Name: Lloyds
Location: 46 Amiens

Put your parsed results into a DataFrame

In [20]:
pubs_df=pd.DataFrame(all_pubs_info)
pubs_df.to_csv("pubs_info.csv", index = False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [15]:
# Get API key from environment variable
YELP_API_KEY = os.environ.get('YELP_API_DATA')

# Ensure API key is available
if YELP_API_KEY is None:
    raise ValueError("Yelp API key not found in environment variables.")
# Read data from CSV
station_df = pd.read_csv("station_details.csv")
yelp_pubs_info = []
#Define URL and parameters
for index, station in station_df.iterrows():
    latitude = station['Latitude']
    longitude = station['Longitude']
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": 1000,
        "categories": "pub", 
        "limit": 30
    }
    headers = {
        "Accept": "application/json",
        "Authorization": 'Bearer ' + YELP_API_KEY
    }
#make request
response = requests.get(yelp_url, params=params, headers=headers)
#check request success 
if response.status_code == 200:
    yelp_data = response.json()
print(response.status_code)
print(station_df)

200
                               Station   Latitude  Longitude  Free Bikes
0                         GEORGES LANE  53.350230  -6.279696           4
1    NORTH CIRCULAR ROAD (O'CONNELL'S)  53.357841  -6.251557          11
2                    PHIBSBOROUGH ROAD  53.356307  -6.273717           4
3                     SMITHFIELD NORTH  53.349562  -6.278198          13
4                       CLONMEL STREET  53.336021  -6.262980          24
..                                 ...        ...        ...         ...
109                  JAMES STREET EAST  53.336597  -6.248109          11
110               PARNELL SQUARE NORTH  53.353742  -6.265301           0
111                FOWNES STREET UPPER  53.344603  -6.263371           6
112                         SMITHFIELD  53.347692  -6.278214          10
113                     FRANCIS STREET  53.342081  -6.275233           9

[114 rows x 4 columns]


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [16]:
# Function to get Yelp data for given latitude/longitude
def get_yelp_data(latitude, longitude, YELP_API_KEY):
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Accept": "application/json",
        "Authorization": 'Bearer ' + YELP_API_KEY 
    }
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": 1000,
        "categories": "pub",
        "limit": 30
    }
    response = requests.get(yelp_url, params=params, headers=headers)

    if response.status_code == 200:
        yelp_data = response.json()
        businesses = yelp_data.get("businesses", [])

        if businesses:
            business = businesses[0]
            return {
                "Name": business.get("name"),
                "Rating": business.get("rating"),
                "Location": {
                    "Address": business.get("location", {}).get("address1"),
                    "City": business.get("location", {}).get("city"),
                    "State": business.get("location", {}).get("state"),
                },
                "Review Count": business.get("review_count", 'N/A'),
                "Price": business.get("price", 'N/A')
            }
        else:
            return {}
    else:
        print(f"Yelp API Error: {response.status_code}")
        print(response.json()) 
        return {}

#results list for Yelp station data
yelp_station_results = []
# keep track of unique pubs
yelp_unique_pubs = set()

# Iterate through Dataframe rows
for index, station in station_df.iterrows():
    latitude = station['Latitude']
    longitude = station['Longitude']
    name = station['Station']
    free_bikes = station['Free Bikes']

    # Get Yelp data for current station
    yelp_data = get_yelp_data(latitude, longitude, YELP_API_KEY)

    # Extract relevant Yelp information
    name_yelp = yelp_data.get('Name')
    
    if name_yelp not in yelp_unique_pubs:
        location_yelp = yelp_data.get('Location', {})
        rating_yelp = yelp_data.get('Rating', 'N/A')
        review_count_yelp = yelp_data.get('Review Count', 'N/A')
        price_yelp = yelp_data.get('Price', 'N/A')

        # Store information in dictionary
        yelp_station_info = {
            "Station": name,
            "Name": name_yelp,
            "Location": location_yelp,
            "Free Bikes": free_bikes,
            "Rating": rating_yelp,
            "Review Count": review_count_yelp,
            "Price": price_yelp
        }

        # Append dictionary information to above list
        yelp_station_results.append(yelp_station_info)
        yelp_unique_pubs.add(name_yelp)
# Create DataFrame from results list
yelp_df = pd.DataFrame(yelp_station_results)
# Print DataFrame
print(yelp_df)

                              Station                              Name  \
0                        GEORGES LANE                 L Mulligan Grocer   
1   NORTH CIRCULAR ROAD (O'CONNELL'S)                    Wallace's Asti   
2                      CLONMEL STREET                 Las Tapas de Lola   
3                  MOUNT STREET LOWER  The Schoolhouse Bar & Restaurant   
4                  CHRISTCHURCH PLACE                     Darkey Kellys   
5                     GRANTHAM STREET                   Camden Exchange   
6                       PEARSE STREET                   The Boxty House   
7                         EXCISE WALK                  J2 Grill & Sushi   
8             FITZWILLIAM SQUARE WEST                 Matt The Thresher   
9                     PORTOBELLO ROAD                          Richmond   
10             FREDERICK STREET SOUTH                     The Pig's Ear   
11                       HANOVER QUAY                  The Marker Hotel   
12                 OLIVER

Put your parsed results into a DataFrame

In [17]:
# Create DataFrame results
yelp_df = pd.DataFrame(yelp_station_results)
# Print DataFrame
print(yelp_df)
# Save DataFrame to CSV file
yelp_df.to_csv("yelp_results.csv", index=False)

                              Station                              Name  \
0                        GEORGES LANE                 L Mulligan Grocer   
1   NORTH CIRCULAR ROAD (O'CONNELL'S)                    Wallace's Asti   
2                      CLONMEL STREET                 Las Tapas de Lola   
3                  MOUNT STREET LOWER  The Schoolhouse Bar & Restaurant   
4                  CHRISTCHURCH PLACE                     Darkey Kellys   
5                     GRANTHAM STREET                   Camden Exchange   
6                       PEARSE STREET                   The Boxty House   
7                         EXCISE WALK                  J2 Grill & Sushi   
8             FITZWILLIAM SQUARE WEST                 Matt The Thresher   
9                     PORTOBELLO ROAD                          Richmond   
10             FREDERICK STREET SOUTH                     The Pig's Ear   
11                       HANOVER QUAY                  The Marker Hotel   
12                 OLIVER

PermissionError: [Errno 13] Permission denied: 'yelp_results.csv'

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The yelp api is good for getting more information to work with regarding pub information. However the foursquare api provides a larger list of pubs.

Get the top 10 restaurants according to their rating

In [1]:
#sort restauraunts by rating to find top 10 
top_10= yelp_df.sort_values(by="Rating", ascending=False)
print(top_10)

NameError: name 'yelp_df' is not defined